# Geocoding
## 使用データのロード

In [2]:
import os
print(os.getcwd())

c:\Users\hkato\Desktop\MHLW\RDD\data\tukuba-geocode\code


In [5]:
import pandas as pd
post_list = pd.read_csv("../raw/MLIT/08220_2019.csv", encoding="SHIFT-JIS")
post_list

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,8,茨城県,8220,つくば市,82200001000,赤塚,36.042677,140.123150,0,1
1,8,茨城県,8220,つくば市,82200002000,明石,36.182065,140.053737,0,1
2,8,茨城県,8220,つくば市,82200003000,旭,36.112890,140.072495,0,1
3,8,茨城県,8220,つくば市,82200004000,安食,36.167751,140.016345,0,1
4,8,茨城県,8220,つくば市,82200005000,あしび野,35.960604,140.112185,0,1
...,...,...,...,...,...,...,...,...,...,...
294,8,茨城県,8220,つくば市,82200234000,中東,36.104534,140.063642,0,1
295,8,茨城県,8220,つくば市,82200235000,酒丸,36.097668,140.058749,0,1
296,8,茨城県,8220,つくば市,82200236000,上河原崎下河原崎入会地,36.072624,140.038812,0,1
297,8,茨城県,8220,つくば市,82200237000,上河原崎元宮本,36.075704,140.036447,0,1


In [6]:
admin_list = pd.read_csv("../raw/tukuba-admin.csv", encoding="SHIFT-JIS")
admin_list

,name,address,zip,latitude,longitude
0,大穂保健センター,茨城県つくば市筑穂1-10-4,3003257,36.137015,140.082554
1,桜保健センター,茨城県つくば市流星台61-1,3050008,36.095188,140.128661
2,谷田部保健センター,茨城県つくば市谷田部4774-18,3050861,36.037118,140.074604
3,健康増進課,茨城県つくば市研究学園1-1-1,3058555,36.083653,140.076709


In [17]:
admin_list["label"] = ["oho", "sakura", "yatabe", "hall"]
admin_list

,name,address,zip,latitude,longitude,label
0,大穂保健センター,茨城県つくば市筑穂1-10-4,3003257,36.137015,140.082554,oho
1,桜保健センター,茨城県つくば市流星台61-1,3050008,36.095188,140.128661,sakura
2,谷田部保健センター,茨城県つくば市谷田部4774-18,3050861,36.037118,140.074604,yatabe
3,健康増進課,茨城県つくば市研究学園1-1-1,3058555,36.083653,140.076709,hall


## `GeoPy`による直線距離の計算

`geopy.distance`を用いる。計算方法は二種類

1. 測地距離（`import geodesic`）：楕円である地球上で、2ポイント間の最短距離を結んだ距離
1. 大円距離（`import great_circle`）：球面上の2点間の長さが最短となる距離（早いが精度悪い）

測地距離を使用する

In [36]:
from geopy.distance import geodesic

for i in range(len(admin_list)):
    to = ((admin_list.at[i, "latitude"]), (admin_list.at[i, "longitude"]))
    label = "dist_" + admin_list.at[i, "label"]
    for j in range(len(post_list)):
        start = ((post_list.at[j, "緯度"]), (post_list.at[j, "経度"]))
        x = geodesic(to, start).km
        post_list.at[j, label] = x

post_list

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード,dist_oho,dist_sakura,dist_yatabe,dist_hall
0,8,茨城県,8220,つくば市,82200001000,赤塚,36.042677,140.123150,0,1,11.087947,5.847745,4.418146,6.178854
1,8,茨城県,8220,つくば市,82200002000,明石,36.182065,140.053737,0,1,5.631338,11.764594,16.192805,11.113957
2,8,茨城県,8220,つくば市,82200003000,旭,36.112890,140.072495,0,1,2.825992,5.425518,8.409804,3.266250
3,8,茨城県,8220,つくば市,82200004000,安食,36.167751,140.016345,0,1,6.865235,12.924460,15.415237,10.798468
4,8,茨城県,8220,つくば市,82200005000,あしび野,35.960604,140.112185,0,1,19.755827,15.007029,9.141176,14.022946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,8,茨城県,8220,つくば市,82200234000,中東,36.104534,140.063642,0,1,3.986087,5.946085,7.545365,2.598652
295,8,茨城県,8220,つくば市,82200235000,酒丸,36.097668,140.058749,0,1,4.863688,6.301852,6.868758,2.243778
296,8,茨城県,8220,つくば市,82200236000,上河原崎下河原崎入会地,36.072624,140.038812,0,1,8.158655,8.470983,5.091299,3.626308
297,8,茨城県,8220,つくば市,82200237000,上河原崎元宮本,36.075704,140.036447,0,1,7.969882,8.582151,5.490930,3.732243
